# 🗄️ Tutorial Completo: HBase + Cassandra

**🎯 Objetivos:**
- Conectarse a HBase y realizar operaciones básicas
- Conectarse a Cassandra y crear keyspaces/tablas
- Realizar operaciones CRUD en ambas bases NoSQL
- Integrar con Spark para análisis de datos

**🔧 Servicios necesarios:**
- HBase: `http://localhost:16010` (Web UI)
- Cassandra: Puerto `9042` (CQL)
- Spark: Integración con ambas bases


## 📦 Instalación de Drivers Python


In [1]:
# Instalar drivers necesarios
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado exitosamente")
    except Exception as e:
        print(f"❌ Error instalando {package}: {e}")

# Drivers necesarios
packages = [
    "happybase",        # HBase Python client
    "cassandra-driver", # Cassandra Python driver
    "thrift",          # Dependencia para HBase
    "pandas",          # Para manipulación de datos
]

print("🔧 Instalando drivers NoSQL...")
for package in packages:
    install_package(package)

print("\n🎉 ¡Instalación completada!")


🔧 Instalando drivers NoSQL...
Defaulting to user installation because normal site-packages is not writeable
✅ happybase instalado exitosamente
Defaulting to user installation because normal site-packages is not writeable
✅ cassandra-driver instalado exitosamente
Defaulting to user installation because normal site-packages is not writeable
✅ thrift instalado exitosamente
Defaulting to user installation because normal site-packages is not writeable
✅ pandas instalado exitosamente

🎉 ¡Instalación completada!


## 🏛️ PARTE 1: HBase (Columnar NoSQL)

**HBase** es una base de datos **columnar** distribuida, ideal para:
- Grandes volúmenes de datos
- Acceso aleatorio rápido
- Esquemas flexibles
- Integración con Hadoop


In [2]:
# 🔗 Conectar a HBase
import happybase
import time
from datetime import datetime

try:
    # Conectar al contenedor HBase
    hbase_connection = happybase.Connection(
        host='hbase',  # Nombre del servicio en Docker
        port=9090,     # Puerto Thrift
        timeout=10000
    )
    
    # Test de conexión
    tables = list(hbase_connection.tables())
    
    print("🎉 ¡Conexión a HBase exitosa!")
    print(f"📊 Tablas existentes: {len(tables)}")
    
    if tables:
        print("📋 Lista de tablas:")
        for table in tables:
            print(f"   - {table.decode('utf-8')}")
    else:
        print("📝 No hay tablas creadas aún")
        
except Exception as e:
    print(f"❌ Error conectando a HBase: {e}")
    print("🔍 Verificar que el servicio HBase esté ejecutándose")
    hbase_connection = None


🎉 ¡Conexión a HBase exitosa!
📊 Tablas existentes: 2
📋 Lista de tablas:
   - educacionit_clientes
   - test_table


In [3]:
# 🏗️ Crear tabla en HBase
if hbase_connection:
    try:
        table_name = 'educacionit_clientes'
        
        # Definir familias de columnas
        families = {
            'info': dict(),      # Información personal
            'contact': dict(),   # Información de contacto
            'stats': dict()      # Estadísticas
        }
        
        # Eliminar tabla si existe (para demo)
        if table_name.encode() in hbase_connection.tables():
            print(f"🗑️ Eliminando tabla existente: {table_name}")
            hbase_connection.delete_table(table_name, disable=True)
        
        # Crear nueva tabla
        print(f"🏗️ Creando tabla: {table_name}")
        hbase_connection.create_table(table_name, families)
        
        # Obtener referencia a la tabla
        hbase_table = hbase_connection.table(table_name)
        
        print(f"✅ Tabla '{table_name}' creada exitosamente")
        print(f"👥 Familias de columnas: {list(families.keys())}")
        
    except Exception as e:
        print(f"❌ Error creando tabla: {e}")
        hbase_table = None
else:
    print("⚠️ No hay conexión a HBase")
    hbase_table = None


🗑️ Eliminando tabla existente: educacionit_clientes
🏗️ Creando tabla: educacionit_clientes
✅ Tabla 'educacionit_clientes' creada exitosamente
👥 Familias de columnas: ['info', 'contact', 'stats']


In [4]:
# 📝 Insertar datos en HBase
if hbase_table:
    try:
        print("📝 Insertando datos de ejemplo...")
        
        # Datos de ejemplo
        clientes_data = [
            {
                'row_key': 'cliente_001',
                'data': {
                    b'info:nombre': b'Juan Perez',
                    b'info:edad': b'35',
                    b'info:ciudad': b'Buenos Aires',
                    b'contact:email': b'juan.perez@email.com',
                    b'contact:telefono': b'+54-11-1234-5678',
                    b'stats:compras_total': b'15',
                    b'stats:monto_total': b'25000.50'
                }
            },
            {
                'row_key': 'cliente_002',
                'data': {
                    b'info:nombre': b'Maria Garcia',
                    b'info:edad': b'28',
                    b'info:ciudad': b'Cordoba',
                    b'contact:email': b'maria.garcia@email.com',
                    b'contact:telefono': b'+54-351-9876-5432',
                    b'stats:compras_total': b'8',
                    b'stats:monto_total': b'12750.00'
                }
            }
        ]
        
        # Insertar cada cliente
        for cliente in clientes_data:
            hbase_table.put(cliente['row_key'], cliente['data'])
            print(f"   ✅ Cliente insertado: {cliente['row_key']}")
        
        print(f"\n🎉 ¡{len(clientes_data)} clientes insertados exitosamente!")
        
    except Exception as e:
        print(f"❌ Error insertando datos: {e}")
else:
    print("⚠️ No hay tabla HBase disponible")


📝 Insertando datos de ejemplo...
   ✅ Cliente insertado: cliente_001
   ✅ Cliente insertado: cliente_002

🎉 ¡2 clientes insertados exitosamente!


In [5]:
# 🔍 Consultar datos en HBase
if hbase_table:
    try:
        print("🔍 Consultando datos de HBase...\n")
        
        # 1. Consultar un cliente específico
        print("1️⃣ Consulta por Row Key específica:")
        cliente_001 = hbase_table.row(b'cliente_001')
        
        if cliente_001:
            print("   📋 Datos de cliente_001:")
            for key, value in cliente_001.items():
                family_column = key.decode('utf-8')
                data_value = value.decode('utf-8')
                print(f"      {family_column}: {data_value}")
        
        print("\n" + "="*50)
        
        # 2. Escanear toda la tabla
        print("2️⃣ Escaneo completo de la tabla:")
        
        count = 0
        for row_key, data in hbase_table.scan():
            count += 1
            print(f"\n   👤 Cliente: {row_key.decode('utf-8')}")
            
            # Mostrar datos organizados
            for key, value in data.items():
                family_column = key.decode('utf-8')
                data_value = value.decode('utf-8')
                print(f"      {family_column}: {data_value}")
        
        print(f"\n🎉 Escaneados {count} clientes exitosamente")
        
    except Exception as e:
        print(f"❌ Error consultando datos: {e}")
else:
    print("⚠️ No hay tabla HBase disponible")


🔍 Consultando datos de HBase...

1️⃣ Consulta por Row Key específica:
   📋 Datos de cliente_001:
      contact:email: juan.perez@email.com
      contact:telefono: +54-11-1234-5678
      info:ciudad: Buenos Aires
      info:edad: 35
      info:nombre: Juan Perez
      stats:compras_total: 15
      stats:monto_total: 25000.50

2️⃣ Escaneo completo de la tabla:

   👤 Cliente: cliente_001
      contact:email: juan.perez@email.com
      contact:telefono: +54-11-1234-5678
      info:ciudad: Buenos Aires
      info:edad: 35
      info:nombre: Juan Perez
      stats:compras_total: 15
      stats:monto_total: 25000.50

   👤 Cliente: cliente_002
      contact:email: maria.garcia@email.com
      contact:telefono: +54-351-9876-5432
      info:ciudad: Cordoba
      info:edad: 28
      info:nombre: Maria Garcia
      stats:compras_total: 8
      stats:monto_total: 12750.00

🎉 Escaneados 2 clientes exitosamente


## 🌟 PARTE 2: Cassandra (Distribuida NoSQL)

**Cassandra** es una base de datos **distribuida** NoSQL, ideal para:
- Alta disponibilidad
- Escalabilidad horizontal
- Tolerancia a fallos
- Escrituras masivas


In [ ]:
# 🔗 Conectar a Cassandra
from cassandra.cluster import Cluster
import uuid

try:
    # Crear cluster de Cassandra
    cluster = Cluster(
        contact_points=['cassandra'],  # Nombre del servicio en Docker
        port=9042
    )
    
    # Crear sesión
    cassandra_session = cluster.connect()
    
    # Test de conexión
    result = cassandra_session.execute("SELECT release_version FROM system.local")
    version = result.one()[0]
    
    print("🎉 ¡Conexión a Cassandra exitosa!")
    print(f"🔢 Versión de Cassandra: {version}")
    
    # Listar keyspaces existentes
    keyspaces = cassandra_session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
    
    print("🗄️ Keyspaces existentes:")
    for keyspace in keyspaces:
        print(f"   - {keyspace[0]}")
        
except Exception as e:
    print(f"❌ Error conectando a Cassandra: {e}")
    print("🔍 Verificar que el servicio Cassandra esté ejecutándose")
    cassandra_session = None
    cluster = None


In [ ]:
# 🏗️ Crear Keyspace y Tablas en Cassandra
if cassandra_session:
    try:
        keyspace_name = 'educacionit_nosql'
        
        # Crear keyspace
        create_keyspace_cql = f"""
        CREATE KEYSPACE IF NOT EXISTS {keyspace_name}
        WITH replication = {{
            'class': 'SimpleStrategy',
            'replication_factor': 1
        }}
        """
        
        cassandra_session.execute(create_keyspace_cql)
        print(f"✅ Keyspace '{keyspace_name}' creado/verificado")
        
        # Usar el keyspace
        cassandra_session.set_keyspace(keyspace_name)
        print(f"🎯 Usando keyspace: {keyspace_name}")
        
        # Crear tabla de productos
        create_productos_cql = """
        CREATE TABLE IF NOT EXISTS productos (
            id UUID PRIMARY KEY,
            nombre TEXT,
            categoria TEXT,
            precio DECIMAL,
            stock INT,
            fecha_creacion TIMESTAMP,
            activo BOOLEAN
        )
        """
        
        cassandra_session.execute(create_productos_cql)
        print("✅ Tabla 'productos' creada")
        
        # Crear tabla de ventas
        create_ventas_cql = """
        CREATE TABLE IF NOT EXISTS ventas (
            fecha_venta DATE,
            id_venta UUID,
            cliente_id INT,
            producto_id UUID,
            cantidad INT,
            precio_unitario DECIMAL,
            total DECIMAL,
            PRIMARY KEY (fecha_venta, id_venta)
        ) WITH CLUSTERING ORDER BY (id_venta ASC)
        """
        
        cassandra_session.execute(create_ventas_cql)
        print("✅ Tabla 'ventas' creada")
        
        print("\n🎉 ¡Keyspace y tablas creados exitosamente!")
        
    except Exception as e:
        print(f"❌ Error creando keyspace/tablas: {e}")
else:
    print("⚠️ No hay conexión a Cassandra")


In [8]:
# 📝 Insertar y Consultar datos en Cassandra
if cassandra_session:
    try:
        print("📝 Insertando datos de ejemplo en Cassandra...\n")
        
        # Insertar productos
        print("1️⃣ Insertando productos...")
        from datetime import datetime, date
        
        productos_data = [
            (uuid.uuid4(), 'Laptop Gaming', 'Electronica', 125000.00, 15, datetime.now(), True),
            (uuid.uuid4(), 'Mouse Inalambrico', 'Accesorios', 2500.50, 50, datetime.now(), True),
            (uuid.uuid4(), 'Teclado Mecanico', 'Accesorios', 8900.00, 25, datetime.now(), True),
        ]
        
        insert_producto_stmt = cassandra_session.prepare(
            "INSERT INTO productos (id, nombre, categoria, precio, stock, fecha_creacion, activo) VALUES (?, ?, ?, ?, ?, ?, ?)"
        )
        
        producto_ids = []
        for producto in productos_data:
            producto_ids.append(producto[0])
            cassandra_session.execute(insert_producto_stmt, producto)
            print(f"   ✅ Producto insertado: {producto[1]}")
        
        # Insertar ventas
        print("\n2️⃣ Insertando ventas...")
        
        ventas_data = [
            (date.today(), uuid.uuid4(), 1001, producto_ids[0], 1, 125000.00, 125000.00),
            (date.today(), uuid.uuid4(), 1002, producto_ids[1], 2, 2500.50, 5001.00),
            (date.today(), uuid.uuid4(), 1003, producto_ids[2], 1, 8900.00, 8900.00),
        ]
        
        insert_venta_stmt = cassandra_session.prepare(
            "INSERT INTO ventas (fecha_venta, id_venta, cliente_id, producto_id, cantidad, precio_unitario, total) VALUES (?, ?, ?, ?, ?, ?, ?)"
        )
        
        for venta in ventas_data:
            cassandra_session.execute(insert_venta_stmt, venta)
            print(f"   ✅ Venta insertada: Cliente {venta[2]} - Total ${venta[6]}")
        
        print(f"\n🎉 ¡Datos insertados exitosamente!")
        
        # CONSULTAR DATOS
        print("\n" + "="*60)
        print("🔍 Consultando datos de Cassandra...\n")
        
        # Consultar productos
        print("📦 Productos disponibles:")
        productos = cassandra_session.execute("SELECT * FROM productos")
        for producto in productos:
            print(f"   • {producto.nombre} - ${producto.precio} (Stock: {producto.stock})")
        
        # Consultar ventas del día
        fecha_hoy = date.today()
        print(f"\n💰 Ventas de hoy ({fecha_hoy}):")
        ventas_hoy = cassandra_session.execute(
            f"SELECT * FROM ventas WHERE fecha_venta = '{fecha_hoy}'"
        )
        
        total_ventas = 0
        count_ventas = 0
        for venta in ventas_hoy:
            count_ventas += 1
            total_ventas += float(venta.total)
            venta_total = float(venta.total)
            print(f"   • Cliente {venta.cliente_id} - Cantidad: {venta.cantidad} - Total: {venta_total:.2f} pesos")
        
        print(f"\n📊 Resumen: {count_ventas} ventas - Total: {total_ventas:.2f} pesos")
        
    except Exception as e:
        print(f"❌ Error con datos Cassandra: {e}")
else:
    print("⚠️ No hay conexión a Cassandra")


📝 Insertando datos de ejemplo en Cassandra...

1️⃣ Insertando productos...
   ✅ Producto insertado: Laptop Gaming
   ✅ Producto insertado: Mouse Inalambrico
   ✅ Producto insertado: Teclado Mecanico

2️⃣ Insertando ventas...
   ✅ Venta insertada: Cliente 1001 - Total $125000.0
   ✅ Venta insertada: Cliente 1002 - Total $5001.0
   ✅ Venta insertada: Cliente 1003 - Total $8900.0

🎉 ¡Datos insertados exitosamente!

🔍 Consultando datos de Cassandra...

📦 Productos disponibles:
   • Laptop Gaming - $125000 (Stock: 15)
   • Laptop Gaming - $125000 (Stock: 15)
   • Teclado Mecanico - $8900 (Stock: 25)
   • Mouse Inalambrico - $2500.5 (Stock: 50)
   • Teclado Mecanico - $8900 (Stock: 25)
   • Laptop Gaming - $125000 (Stock: 15)
   • Mouse Inalambrico - $2500.5 (Stock: 50)
   • Mouse Inalambrico - $2500.5 (Stock: 50)
   • Teclado Mecanico - $8900 (Stock: 25)
   • Laptop Gaming - $125000 (Stock: 15)
   • Mouse Inalambrico - $2500.5 (Stock: 50)
   • Mouse Inalambrico - $2500.5 (Stock: 50)
   • Mo

25/09/20 11:48:51 WARN DataStreamer: Exception for BP-651295859-172.28.1.2-1757645932520:blk_1073742217_1414
java.net.SocketTimeoutException: 70000 millis timeout while waiting for channel to be ready for read. ch : java.nio.channels.SocketChannel[connected local=/172.28.1.7:49288 remote=/172.28.1.3:9866]
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:163)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:161)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:131)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:118)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:519)
	at org.apache.hadoop.hdfs.protocol.datatransfer.PipelineAck.readFields(PipelineAck.java:213)
	at org.apache.hadoop.hdfs.DataStreamer$ResponseProcessor.run(DataStream

## 🔥 PARTE 3: Integración con Spark

**Integrar Spark** con NoSQL para análisis masivos de datos.


In [ ]:
# 🚀 CONFIGURACIÓN SPARK EXITOSA (Copiada de spark_cluster_final)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

print("🔄 Deteniendo sesiones Spark existentes...")
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(3)
except:
    print("🔍 No había sesión activa")

# CONFIGURACIÓN EXACTA QUE FUNCIONA
spark_nosql = SparkSession.builder \
    .appName("NoSQL-Integration-Working") \
    .master("spark://master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .getOrCreate()

spark_nosql.sparkContext.setLogLevel("WARN")

print("🎉 ¡Spark configurado para NoSQL con configuración exitosa!")
print(f"📊 Spark UI: {spark_nosql.sparkContext.uiWebUrl}")
print(f"🔗 Master: {spark_nosql.sparkContext.master}")

# Test rápido
print("\n🧪 Test básico de conectividad:")
test_rdd = spark_nosql.sparkContext.parallelize([1, 2, 3, 4, 5])
suma = test_rdd.reduce(lambda a, b: a + b)
print(f"✅ Test exitoso - Suma: {suma}")

# Ejemplo: Crear DataFrame desde datos de Cassandra
if 'cassandra_session' in locals() and cassandra_session:
    try:
        print("\n📊 Creando DataFrame desde Cassandra...")
        
        # Obtener datos de productos
        productos_rows = cassandra_session.execute("SELECT * FROM productos")
        productos_data = []
        
        for row in productos_rows:
            productos_data.append({
                'id': str(row.id),
                'nombre': row.nombre,
                'categoria': row.categoria,
                'precio': float(row.precio),
                'stock': row.stock,
                'activo': row.activo
            })
        
        if productos_data:
            # Crear DataFrame
            productos_schema = StructType([
                StructField("id", StringType(), True),
                StructField("nombre", StringType(), True),
                StructField("categoria", StringType(), True),
                StructField("precio", DoubleType(), True),
                StructField("stock", IntegerType(), True),
                StructField("activo", BooleanType(), True)
            ])
            
            productos_df = spark_nosql.createDataFrame(productos_data, productos_schema)
            productos_df.createOrReplaceTempView("productos_spark")
            
            print(f"✅ DataFrame creado: {productos_df.count()} productos")
            
            # Análisis con Spark SQL
            print("\n📈 Análisis con Spark:")
            categoria_analysis = spark_nosql.sql("""
                SELECT categoria, 
                       COUNT(*) as cantidad,
                       AVG(precio) as precio_promedio,
                       SUM(stock) as stock_total
                FROM productos_spark 
                WHERE activo = true
                GROUP BY categoria
                ORDER BY precio_promedio DESC
            """)
            
            categoria_analysis.show()
        else:
            print("⚠️ No hay datos de productos en Cassandra")
        
    except Exception as e:
        print(f"❌ Error análisis Spark: {e}")
else:
    print("⚠️ Ejecuta primero las celdas de Cassandra para tener datos")


In [ ]:
# 🧹 Cerrar todas las conexiones
print("🧹 Cerrando conexiones...\n")

# Cerrar conexión HBase
if 'hbase_connection' in locals() and hbase_connection:
    try:
        hbase_connection.close()
        print("✅ Conexión HBase cerrada")
    except:
        print("⚠️ Error cerrando HBase")

# Cerrar conexión Cassandra
if 'cassandra_session' in locals() and cassandra_session:
    try:
        cassandra_session.shutdown()
        print("✅ Sesión Cassandra cerrada")
    except:
        print("⚠️ Error cerrando Cassandra")

if 'cluster' in locals() and cluster:
    try:
        cluster.shutdown()
        print("✅ Cluster Cassandra cerrado")
    except:
        print("⚠️ Error cerrando cluster Cassandra")

# Cerrar Spark
if 'spark_nosql' in locals() and spark_nosql:
    try:
        spark_nosql.stop()
        print("✅ Sesión Spark cerrada")
    except:
        print("⚠️ Error cerrando Spark")

print("\n🎉 ¡Limpieza completada!")


## 📚 Resumen y Comandos Útiles

### 🏛️ **HBase - Comandos Clave:**
```python
# Conectar
connection = happybase.Connection(host='hbase', port=9090)

# Crear tabla
connection.create_table('tabla', {'familia': dict()})

# Insertar datos
table.put('row_key', {b'familia:columna': b'valor'})

# Consultar
row = table.row(b'row_key')
for key, data in table.scan():
    print(key, data)
```

### 🌟 **Cassandra - Comandos CQL:**
```python
# Conectar
cluster = Cluster(['cassandra'])
session = cluster.connect()

# Crear keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS mi_keyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")

# Crear tabla
session.execute("CREATE TABLE IF NOT EXISTS tabla (id UUID PRIMARY KEY, nombre TEXT)")

# Insertar
session.execute("INSERT INTO tabla (id, nombre) VALUES (?, ?)", [uuid.uuid4(), 'valor'])

# Consultar
rows = session.execute("SELECT * FROM tabla")
```

### 🔥 **Spark + NoSQL:**
```python
# Configuración básica
spark = SparkSession.builder \
    .appName("NoSQL-Integration") \
    .master("spark://master:7077") \
    .getOrCreate()

# Crear DataFrame desde datos NoSQL
df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("tabla_temporal")

# Análisis con SQL
result = spark.sql("SELECT * FROM tabla_temporal")
```

### 🌐 **URLs de Monitoreo:**
- **HBase Master:** http://localhost:16010
- **Cassandra:** Puerto 9042 (CQL)
- **Spark UI:** http://localhost:4040 (cuando hay aplicaciones activas)

---

## 🎯 **¡Tutorial Completado!**

Has aprendido a:
- ✅ Conectarte a HBase y realizar operaciones CRUD
- ✅ Trabajar con Cassandra (keyspaces, tablas, consultas)
- ✅ Integrar ambas bases NoSQL con Spark
- ✅ Realizar análisis de datos distribuidos

**¡Ahora tienes las herramientas para trabajar con Big Data NoSQL!** 🚀


In [ ]:
# 🛠️ SOLUCIÓN: Detener sesión problemática y recrear
import time

print("🔄 Deteniendo TODAS las sesiones Spark...")

# Detener sesión actual si existe
try:
    spark_nosql.stop()
    print("🛑 Sesión spark_nosql detenida")
    time.sleep(3)
except:
    print("🔍 No había spark_nosql activa")

# Detener cualquier otra sesión
try:
    spark.stop()
    print("🛑 Sesión spark detenida")
    time.sleep(3)
except:
    print("🔍 No había spark activa")

print("⏳ Esperando liberación de recursos...")
time.sleep(5)

# Recrear con configuración optimizada
print("\n🚀 Creando nueva sesión Spark optimizada...")

spark_nosql = SparkSession.builder \
    .appName("NoSQL-Final-Clean") \
    .master("spark://master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.executor.memory", "800m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .getOrCreate()

spark_nosql.sparkContext.setLogLevel("WARN")

print("🎉 ¡Nueva sesión Spark creada!")
print(f"📊 Spark UI: {spark_nosql.sparkContext.uiWebUrl}")

# Test inmediato
print("\n🧪 Test de conectividad:")
test_rdd = spark_nosql.sparkContext.parallelize([1, 2, 3, 4, 5])
suma = test_rdd.reduce(lambda a, b: a + b)
print(f"✅ Test exitoso - Suma: {suma}")
print("🎯 ¡Sesión funcionando correctamente!")


In [ ]:
# 🎉 CLUSTER LIMPIO - Crear sesión Spark optimizada
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

print("🎯 Creando sesión Spark con cluster limpio...")

# Configuración optimizada para cluster con recursos disponibles
spark_clean = SparkSession.builder \
    .appName("NoSQL-Clean-Session") \
    .master("spark://master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "2") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

spark_clean.sparkContext.setLogLevel("WARN")

print("🎉 ¡Sesión Spark creada exitosamente!")
print(f"📊 Spark UI: {spark_clean.sparkContext.uiWebUrl}")
print(f"🔗 Master: {spark_clean.sparkContext.master}")

# Test inmediato
print("\n🧪 Test de conectividad:")
test_rdd = spark_clean.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 4)
suma = test_rdd.reduce(lambda a, b: a + b)
particiones = test_rdd.getNumPartitions()

print(f"✅ Test exitoso:")
print(f"   🔢 Suma: {suma}")
print(f"   📊 Particiones: {particiones}")
print(f"   🎯 ¡Cluster funcionando perfectamente!")

# Análisis con datos de Cassandra
if 'cassandra_session' in locals() and cassandra_session:
    try:
        print("\n📊 Análisis NoSQL con Spark + Cassandra:")
        
        # Obtener datos de productos
        productos_rows = cassandra_session.execute("SELECT * FROM productos")
        productos_data = []
        
        for row in productos_rows:
            productos_data.append({
                'id': str(row.id),
                'nombre': row.nombre,
                'categoria': row.categoria,
                'precio': float(row.precio),
                'stock': row.stock,
                'activo': row.activo
            })
        
        if productos_data:
            # Crear DataFrame con Spark
            productos_schema = StructType([
                StructField("id", StringType(), True),
                StructField("nombre", StringType(), True),
                StructField("categoria", StringType(), True),
                StructField("precio", DoubleType(), True),
                StructField("stock", IntegerType(), True),
                StructField("activo", BooleanType(), True)
            ])
            
            productos_df = spark_clean.createDataFrame(productos_data, productos_schema)
            productos_df.createOrReplaceTempView("productos_nosql")
            
            print(f"✅ DataFrame creado: {productos_df.count()} productos")
            
            # Análisis distribuido con Spark SQL
            print("\n📈 Análisis por categoría:")
            resultado = spark_clean.sql("""
                SELECT categoria,
                       COUNT(*) as cantidad_productos,
                       ROUND(AVG(precio), 2) as precio_promedio,
                       SUM(stock) as stock_total,
                       ROUND(SUM(precio * stock), 2) as valor_inventario
                FROM productos_nosql 
                WHERE activo = true
                GROUP BY categoria
                ORDER BY valor_inventario DESC
            """)
            
            resultado.show(truncate=False)
            
            # Top productos más caros
            print("\n💰 Top productos más caros:")
            top_productos = spark_clean.sql("""
                SELECT nombre, categoria, precio, stock
                FROM productos_nosql 
                WHERE activo = true
                ORDER BY precio DESC
                LIMIT 5
            """)
            
            top_productos.show(truncate=False)
            
        else:
            print("⚠️ No hay datos de productos en Cassandra")
            
    except Exception as e:
        print(f"❌ Error en análisis: {e}")
else:
    print("⚠️ Ejecuta primero las celdas de Cassandra para tener datos")

print("\n🎉 ¡Análisis NoSQL completado exitosamente!")


## 📋 **ORDEN DE EJECUCIÓN CORRECTO**

Para que el tutorial funcione completamente, ejecuta las celdas en este orden:

### 🔄 **Secuencia Recomendada:**

1. **Celda 2:** 📦 Instalar drivers Python (`happybase`, `cassandra-driver`)
2. **Celda 4:** 🏛️ Conectar a HBase 
3. **Celda 5:** 🏗️ Crear tabla en HBase
4. **Celda 6:** 📝 Insertar datos en HBase
5. **Celda 7:** 🔍 Consultar datos en HBase
6. **Celda 9:** 🌟 Conectar a Cassandra
7. **Celda 10:** 🏗️ Crear Keyspace y Tablas en Cassandra
8. **Celda 11:** 📝 Insertar y Consultar datos en Cassandra ⭐ **IMPORTANTE**
9. **Celda 17:** 🎉 Análisis con Spark (requiere datos de Cassandra)
10. **Celda 14:** 🧹 Cerrar conexiones

### ⚠️ **Nota Importante:**
- La **Celda 17** (Spark + Cassandra) requiere que hayas ejecutado la **Celda 11** primero
- Si no hay datos en Cassandra, verás el mensaje: "⚠️ Ejecuta primero las celdas de Cassandra"

### 🎯 **Ejecuta ahora:**
1. **Celda 11** (si no lo has hecho)
2. **Celda 17** (análisis con Spark)
